# Lesson 1 - GFS 13km Processing
## Overview
In this lesson, we will be running the Python forcing engine code on basin 01447720. We will be processing GFS 13km gaussian grid data to the modeling domain. In this example, no downscaling is taking place.

### Requirements
The easiest and recommended way to run these lessons is via the [wrfhydro/forcing-engine](https://hub.docker.com/r/wrfhydro/forcing-engine/) Docker container, which has all software dependencies and data pre-installed.

* Docker = v.18.09.2 
* Web browser (Google Chrome recommended)

### Where to get help and/or post issues
If you have general questions about Docker, there are ample online resources including the excellent Docker documentation at https://docs.docker.com/.

The best place to ask questions or post issues with these lessons is via our online contact form at https://ral.ucar.edu/projects/wrf_hydro/contact.


## How to run
Make sure you have Docker installed and that it can access your localhost ports. Most out-of-the-box Docker installations accepting all defaults will have this configuration.

**Step 1: Open a terminal or PowerShell session**

**Step 2: Pull the wrfhydro/forcing-engine Docker container**
Each training container is specific to a release version of the NWM source code, which can be found at https://github.com/NCAR/wrf_hydro_nwm_public/releases.

Issue the following command in your terminal to pull a specific version of the training corresponding to your code release version.

`docker pull wrfhydro/forcing-engine:latest`

**Step 3: Start the training container**
Issue the following command in your terminal session to start the training Docker container.

`docker run --name forcing-engine -p 8888:8888 -it wrfhydro/forcing-engine:latest`

The container will start and perform a number of actions before starting the training. 

**Note: If you have already started the training once you will need to remove the previous container using the command

`docker rm forcing-engine`

**Step 4: Connect to Jupyter Notebook server using your browser**

At the end of the container startup process an address and password will be printed to the terminal. The address and password are used to connect to the container Jupyter Notebook server. All training lesson notebooks in this container are in the `/home/docker/wrf-hydro-training/lessons` directory and can be opened in your browser using Jupyter.

## What is included

* The Python forcing engine code. This code is still in Beta testing/development.
* Future code will be updated on the forcing engine repo, and Docker images will be updated accordingly. 
* Input GRIB2 forcing files (GFS) that are ingested into the FE. Note these files have been subsetted to the Pennsylvania region to conserve space. 
* Domain files necessary to process input forcings to the domain. 
* Pre-installed ESMF that was built against MPICH. This includes the Python ESMPy wrapper. 
* Forcing engine training lessons as Jupyter Notebooks
* Jupyter Notebook server.

**Note: Port forwarding is setup with the `-p 8888:8888` argument, which maps your localhost port to the container port. If you already have something running on port 8888 on your localhost you will need to change this number**

## Overview of Input Forcings
We will first examine the input forcing GRIB2 files needed for this exercise.

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Forcing_Inputs
ls

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Forcing_Inputs/GFS_13km
ls

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Forcing_Inputs/GFS_13km/gfs.2019022412
ls

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Forcing_Inputs/GFS_13km/gfs.2019022412
wgrib2 -V -match ":(PRATE):" gfs.t12z.sfluxgrbf24.grib2

**NOTE** - These GRIB2 files were initially subsetted to a lat/lon bounding box using wgrib2's nifty -small_grib flag. It's a great way to slim your GRIB2 inputs to a more manageable size when working on regional/local domains. Documentation on this can be found at - https://www.cpc.ncep.noaa.gov/products/wesley/wgrib2/small_grib.html

## Orientation of Python Code
We will first begin by taking a high-level view of the directory structure above the code that contains our inputs and outputs. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing
ls

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing
ls DOMAIN

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/WrfHydroForcing
ls

**NOTE** - The forcing engine is called by genForcing.py for all instances of the processing. No need for multiple calls to different programs. 

In [ ]:
%%bash
ls /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/WrfHydroForcing/core

## Forcing Engine Configuration File 
We will inspect the configuration file in detail, and outline the options for this specific exercise. 

In [ ]:
%%bash 
ls /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Config

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Config/template_forcing_engine_AnA.config

## Execution of the Forcing Engine
We will now execute the forcing engine, providing the configuration file just discussed. For this exercise, we will use two CPU cores in our MPI execution. Note that in larger domains with more data, you will want to use more CPUs to reduce the memory usage on a given core. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/WrfHydroForcing
mpiexec -np 2 python genForcing.py /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Config/template_forcing_engine_Medium.config

## Inspection of Outputs
The top level output directory contains a subdirectory for each "cycle's" worth of forcing data. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Output/Medium_Config
ls

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Output/Medium_Config/201902241200
ls

In this directory, we see two additional files to the output files:
    1) A LOG file that contains messaging about the status of the forcing processing.
    2) A COMPLETE flag indicating processing for this cycle is complete. 
Let's inspect the LOG file....

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Output/Medium_Config/201902241200
cat LOG*

Now let's inspect one of the output forcing files. You will note the comprehensive level of geospatial metadata included. If the spatial metadata file is not specified in the configuration file. You will see much less geospatial information and the files most likely will not display correctly in GIS applications. 

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/forcing_engine/Pocono_Forcing/Output/Medium_Config/201902241200
ncdump -h 201902241300.LDASIN_DOMAIN1

© UCAR 2019